In [1]:
import os
os.environ["MUJOCO_GL"] = "egl"
from common.parser import parse_cfg
from common.seed import set_seed
from envs import make_env
from envs.myosuite import MYOSUITE_TASKS
from tdmpc2 import TDMPC2
from pathlib import Path
from omegaconf import OmegaConf

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.tensor_utils as TensorUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
from robomimic.envs.env_base import EnvBase
from robomimic.algo import RolloutPolicy

import h5py
import json
import hydra

/home/krishnans/miniconda3/envs/robomimic/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/krishnans/miniconda3/envs/robomimic/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
list(MYOSUITE_TASKS.keys())

['myo-reach',
 'myo-reach-hard',
 'myo-pose',
 'myo-pose-hard',
 'myo-obj-hold',
 'myo-obj-hold-hard',
 'myo-key-turn',
 'myo-key-turn-hard',
 'myo-pen-twirl',
 'myo-pen-twirl-hard']

In [3]:
ckpt_dir = Path("/home/krishnans/ngc/tdmpc2/ckpts/myosuite")
data_dir = Path("/home/krishnans/multi_task_experts/datasets/myo10/")
for task in list(MYOSUITE_TASKS.keys()):
    task_name = "-".join([task.split("-")[0], "hand"] + task.split("-")[1:])
    ckpt = ckpt_dir / f"{task_name}-1.pt" 
    overrides = ["hydra/launcher=basic"]
    overrides.append(f"task={task}")
    overrides.append(f"checkpoint={str(ckpt.resolve())}")

    with hydra.initialize():
        cfg = hydra.compose("config.yaml", overrides=overrides)

    dataset_path = list(data_dir.rglob(f"{task}_*.hdf5"))[0]
    assert dataset_path.exists()
    with h5py.File(str(dataset_path), "a") as f:
        print(f"writing env args to dataset: {str(dataset_path)}") 
        env_kwargs = OmegaConf.to_container(cfg, resolve=True)
        env_kwargs["pad_to_shape"] = (115,)
        env_args = {"env_name": MYOSUITE_TASKS[task], "type": 4, "env_kwargs": env_kwargs}
        break
        f["data"].attrs["env_args"] = json.dumps(env_args)


writing env args to dataset: /home/krishnans/multi_task_experts/datasets/myo10/myo-reach/myo-reach_515.hdf5


/tmp/ipykernel_689835/3445706401.py:10: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize():
/tmp/ipykernel_689835/3445706401.py:10: UserWarning: config_path is not specified in hydra.initialize().
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_hydra_main_config_path for more information.
  with hydra.initialize():


In [4]:
env = EnvUtils.create_env_from_metadata(env_args)

MyoSuite:> Registering Myo Envs
    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    
Created environment with name myoHandReachFixed-v0
Action size is 39
ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)


In [5]:
ckpt_path = "/home/krishnans/ngc/robomimic/act_myo10/models/model_epoch_280.pth"

policy, ckpt_dict = FileUtils.policy_from_checkpoint(
    ckpt_path=ckpt_path, 
    device='cuda:0', 
    verbose=True
)

============= Loaded Config =============
{
    "algo_name": "act",
    "experiment": {
        "name": "test-myo-act",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": false,
            "wandb_proj_name": "debug"
        },
        "mse": {
            "enabled": false,
            "every_n_epochs": 50,
            "on_save_ckpt": true,
            "num_samples": 20,
            "visualize": true
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 40,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 500,
        "validation_epoch_every_n_steps": 10,
        "env": null,
        "additional_envs": null,
        "render": false,
        "render_video": true,
        "k

/home/krishnans/miniconda3/envs/robomimic/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/krishnans/miniconda3/envs/robomimic/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Use VQ: True, 512, 64
number of parameters: 117.54M
============= Loaded Policy =============
ACT (
  ModuleDict(
    (policy): DETRVAE(
      (transformer): Transformer(
        (encoder): TransformerEncoder(
          (layers): ModuleList(
            (0-3): 4 x TransformerEncoderLayer(
              (self_attn): MultiheadAttention(
                (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
              )
              (linear1): Linear(in_features=512, out_features=3200, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (linear2): Linear(in_features=3200, out_features=512, bias=True)
              (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (dropout1): Dropout(p=0.1, inplace=False)
              (dropout2): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (decoder): Transfo